In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import unicodedata
import re

In [ ]:
import torchvision

In [ ]:
df_train = pd.read_csv('./processed_data/poswords.csv')
# df_train = df_train.sample(frac=1)

In [ ]:
df_label = pd.read_csv('./train_org.csv',usecols=['is_duplicate'])

In [ ]:
df_train_pos = pd.read_csv('./processed_data/poswords_pos.csv')

In [ ]:
df_train.columns

In [ ]:
texts_1=[]
texts_2 =[]
labels = []
for q1,q2,is_dup in zip(df_train.question1_pos,df_train.question2_pos,df_label.is_duplicate):
    texts_1.append(q1.strip('[]').split(', '))
    texts_2.append(q2.strip('[]').split(', '))
    labels.append(int(is_dup))

In [ ]:
pos_1 =[]
pos_2 =[]
for q1,q2 in zip(df_train_pos.question1_pos,df_train_pos.question2_pos):
    pos_1.append(q1.strip('[]').split(', '))
    pos_2.append(q2.strip('[]').split(', '))

In [ ]:
for i in range(len(pos_1)):
    try:
        assert len(texts_1[i])==len(pos_1[i])
        assert len(texts_2[i])==len(pos_2[i])
    except:
        print i
        break
#     assert len(texts_2[i])==len(pos_2[i])

In [ ]:
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300

In [ ]:
from collections import Counter
import itertools
from collections import Counter
import itertools
def build_vocab(sentences, max_vocab=30000):
    word_counts = Counter(itertools.chain(*sentences))
    vocabulary_inv = []
    vocabulary_inv.append("<PAD/>")
    vocabulary_inv.append("<mino/>")
    vocabulary_inv.extend([x[0] for x in word_counts.most_common(max_vocab)])
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

In [ ]:
import numpy as np
vocabulary, vocabulary_inv = build_vocab(test_texts_1+test_texts_2+texts_1+texts_2, max_vocab=200000)

In [ ]:
vocabulary_pos, vocabulary_inv_pos = build_vocab(pos_1+pos_2, max_vocab=200000)

In [ ]:
# import pickle
# output = open('vacabulary_quora_pt.pkl', 'wb')
# pickle.dump(vocabulary,output)
# output.close()
# output = open('vocabulary_inv_quora_pt.pkl', 'wb') 
# pickle.dump(vocabulary,output)
# output.close()

In [ ]:
import pickle
output = open('vacabulary_quora_pt.pkl', 'r')
vocabulary = pickle.load(output)

In [ ]:
def build_data(ques1,ques2,label,vocabulary):
    mino_idx = vocabulary["<mino/>"]
    data_index = []
    for q1,q2,dup in zip(ques1,ques2,label):
        sentence1 = [vocabulary.get(word, mino_idx) for word in q1]
        sentence2 = [vocabulary.get(word, mino_idx) for word in q2]
        data_index.append([sentence1,sentence2,dup])
    return data_index
data_index = build_data(texts_1,texts_2,labels,vocabulary)

In [ ]:
data_pos_index = build_data(pos_1,pos_2,labels,vocabulary_pos)

In [ ]:
for i in range(1000):
    assert len(data_index[i][1])==len(data_pos_index[i][1])

In [ ]:
import torch.nn.init as init

In [ ]:
from gensim.models import KeyedVectors
import numpy as np
EMBEDDING_FILE = '/home/zhangjinbin/research/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,binary=True)
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
nb_words = min(MAX_NB_WORDS, len(vocabulary))+1
embedding_matrix = np.random.normal(loc=0,scale=0.1,size=[nb_words, EMBEDDING_DIM])
for word, i in vocabulary.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [ ]:
del word2vec

In [ ]:
np.shape(embedding_matrix)[0]

In [ ]:
class biRNN(nn.Module):
    def __init__(self,vocab_size,embedding_size,hidden_size,output_size,n_layers = 2):
        super(biRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.output_size =output_size
        self.embed= nn.Embedding(vocab_size,embedding_size)
        self.gru= nn.GRU(input_size=self.embedding_size,
                            hidden_size=self.hidden_size,
                            num_layers = self.n_layers,
                            batch_first =True,
                            bidirectional = True)
        self.da = 200
        self.r = 200
        self.fc1 = nn.Linear(100*4, 1200)
        self.fc2 = nn.Linear(1200, 2)
        self.fcWs1 = nn.Linear(self.hidden_size*2, self.da)
        self.fcWs2 = nn.Linear(self.da,self.r)
        
        self.conv1_3 = nn.Conv2d(1, 200,(3, 300), stride=(1, 300),padding=(1,0))
#         self.conv1_5 = nn.Conv2d(1, 100,(5, 300), stride=(1, 300),padding=(1,0))
#         self.conv1_7 = nn.Conv2d(1, 100,(7, 300), stride=(1, 300),padding=(2,0))
#         self.conv2_3 = nn.Conv2d(1, 200,(3, 300), stride=(1, 300),padding=(1,0))
#         self.conv2_5 = nn.Conv2d(1, 200,(5, 300), stride=(1, 300),padding=(1,0))
        self.conv2_7 = nn.Conv2d(1, 200,(3, 300), stride=(1, 300),padding=(1,0))
#         self.conv2_g = nn.Conv2d(1, 100,(3, 100), stride=(1, 100),padding=(1,0))
        
        self.fc_dedim_mul = nn.Linear(self.da*self.r,100)
        self.fc_dedim_sub = nn.Linear(self.da*self.r,100)
        self.fc_dedim_rep = nn.Linear(self.da*self.r,100)
        #self.fc_gate = nn.Linear(self.hidden_size*2*self.r,self.r*self.hidden_size*2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
    def conv_block(self,input_seq):
        h1_rep3 = self.conv1_3(input_seq.unsqueeze(1)).transpose(1,3).squeeze(1)
        h1_rep3_g = self.conv2_7(input_seq.unsqueeze(1)).squeeze(3)
        h2_rep3 = torch.bmm(self.sigmoid(h1_rep3_g), h1_rep3)
        
#         h1_rep5 = self.self.conv1_5(input_seq.unsqueeze(1)).transpose(1,3).squeeze(1)
#         h1_rep5_g = self.conv1_5(input_seq.unsqueeze(1)).squeeze(3)
#         h2_rep5 = torch.bmm(self.sigmoid(h1_rep1_g), h1_rep1)
        
#         h1_rep7 = self.conv1_3(input_seq.unsqueeze(1)).transpose(1,3).squeeze(1)
#         h1_rep7_g = self.conv2(input_seq.unsqueeze(1)).squeeze(3)
#         h2_rep7 = torch.bmm(self.sigmoid(h1_rep1_g), h1_rep1)
        return h2_rep3
    def forward(self,sentence1,sentence2,length1,length2,pad1_V,pad2_V,cuda_available = False):
        
#         h1 = torch.mean(self.embed(sentence1),1)
#         h1 =torch.squeeze(h1, dim=1
#         h2 = torch.mean(self.embed(sentence2),1)
#         h2 =torch.squeeze(h2,dim=1)

        input_1 = self.embed(sentence1)
        input_2 = self.embed(sentence2)
        if cuda_available:
            output_1 = torch.mul(pad1_V.float().cuda(),input_1)
            output_2 = torch.mul(pad2_V.float().cuda(),input_2)
        else:
            output_1 = torch.mul(pad1_V.float(),input_1)
            output_2 = torch.mul(pad2_V.float(),input_2)
        represent1 = self.conv_block(output_1)
        represent2 = self.conv_block(output_2)
#        h1_rep1 = self.conv2_7(output_1.unsqueeze(1)).transpose(1,3).squeeze(1)
#         h1_rep1_g = self.conv2(output_1.unsqueeze(1)).squeeze(3)#.transpose(1,3)
#         h1_rep1 = torch.mul(h1_rep1, self.sigmoid(h1_rep1_g))
        
#         h2_rep1 = self.conv2(h1_rep1).transpose(1,3).squeeze(1)
#         h2_rep1_g = self.conv2_g(h1_rep1).squeeze(3)
#         h2_rep1 = torch.bmm(self.sigmoid(h2_rep1_g), h2_rep1)
#         represent1 = h2_rep1
        
#         h1_rep2 = self.conv1(output_2.unsqueeze(1)).transpose(1,3)        
#         h1_rep2_g = self.conv1_g(output_2.unsqueeze(1)).transpose(1,3)
#         h1_rep2 = torch.mul(h1_rep2, self.sigmoid(h1_rep2_g))
        
#         h2_rep2 = self.conv2(h1_rep2).transpose(1,3).squeeze(1)
#         h2_rep2_g = self.conv2_g(h1_rep2).squeeze(3)
#         h2_rep2 = torch.bmm(self.sigmoid(h2_rep2_g), h2_rep2)
        
#         out1 = self.conv2(h1.view(input_1.size()[0],1,input_1.size()[1],input_1.size()[2])).squeeze(3)
#         out1_g = self.relu(self.conv1_g(output_1.view(input_1.size()[0],1,input_1.size()[1],input_1.size()[2])))
#         out1_g = out1_g.transpose(1,3).squeeze(1)
#         represent1 = torch.bmm(out1, out1_g) 
        
#         out2 = self.conv1(output_2.view(input_2.size()[0],1,input_2.size()[1],input_2.size()[2])).squeeze(3)
#         out2_g = self.relu(self.conv1_g(output_2.view(input_2.size()[0],1,input_2.size()[1],input_2.size()[2])))
#         out2_g = out2_g.transpose(1,3).squeeze(1)
#         represent2 = torch.bmm(out2, out2_g) 
        
        output1_new = represent1.view(-1,40000)
        output2_new = represent2.view(-1,40000)
        mul_represent = torch.mul(output1_new,output2_new)
        sub_represent = torch.abs(output1_new-output2_new)
        mul_dedim = self.fc_dedim_mul(mul_represent)
        sub_dedim = self.fc_dedim_sub(sub_represent)
        rep1_dedim = self.fc_dedim_rep(output1_new)
        rep2_dedim = self.fc_dedim_rep(output2_new)


#         gate_1 = self.fc_gate(output1_new)
#         gate_2 = self.fc_gate(output2_new)
        
#         re2 = torch.mul(output2_new,self.relu(gate_1))
#         re1 = torch.mul(output1_new,self.relu(gate_2))
#         all_rep_mul = torch.mul(re2,re1)
        

        
#         mul_dedim = self.fc_dedim_mul(all_rep_mul)
        
        out_mul = self.relu(torch.cat((sub_dedim,mul_dedim,rep1_dedim,rep2_dedim),1))
        logits  = self.fc2(self.relu(self.fc1(out_mul)))
        return logits
    def init_weights(self,pretrained_weight):
        #init(self.embed.weight.data)
        initrange = 0.001
#         nn.init.uniform(self.embedding,a=0,b=0.00
        self.fc1.bias.data.fill_(0)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        #torch.nn.init.xavier_uniform(self.fc.weight.data, gain=math.sqrt(2.0))
        #self.embed.weight.data.normal_(mean=0, std=0.01)
        self.embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.fill_(0)
        
        self.fcWs1.weight.data.uniform_(-initrange, initrange)
        self.fcWs1.bias.data.fill_(0)
        self.fcWs2.weight.data.uniform_(-initrange, initrange)
        self.fcWs2.bias.data.fill_(0)
    def initHidden(self,batch_size):
        h0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
        c0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
        return h0,c0

In [ ]:
class biRNN_att(nn.Module):
    def __init__(self,vocab_size,embedding_size,hidden_size,output_size,n_layers = 2):
        super(biRNN_att, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.output_size =output_size
        self.embed= nn.Embedding(vocab_size,embedding_size)
        self.embed_pos= nn.Embedding(len(vocabulary_pos),embedding_size)
        self.gru= nn.GRU(input_size = self.embedding_size,
                            hidden_size = self.hidden_size,
                            num_layers = self.n_layers,
                            batch_first =True,
                            bidirectional = True)
        self.da = 200
        self.r = 200
        self.fc1 = nn.Linear(100*4, 1200)
        self.fc2 = nn.Linear(1200, 2)
        self.fcWs1 = nn.Linear(self.hidden_size*2, self.da)
        self.fcWs2 = nn.Linear(self.da,self.r)
        
        self.conv1_3 = nn.Conv2d(1, 200,(3, 300), stride=(1, 300),padding=(1,0))
#         self.conv1_5 = nn.Conv2d(1, 100,(5, 300), stride=(1, 300),padding=(1,0))
#         self.conv1_7 = nn.Conv2d(1, 100,(7, 300), stride=(1, 300),padding=(2,0))
#         self.conv2_3 = nn.Conv2d(1, 200,(3, 300), stride=(1, 300),padding=(1,0))
#         self.conv2_5 = nn.Conv2d(1, 200,(5, 300), stride=(1, 300),padding=(1,0))
        self.conv2_7 = nn.Conv2d(1, 200,(3, 300), stride=(1, 300),padding=(1,0))
#         self.conv2_g = nn.Conv2d(1, 100,(3, 100), stride=(1, 100),padding=(1,0))
        
        self.fc_dedim_mul = nn.Linear(self.da*self.r,100)
        self.fc_dedim_sub = nn.Linear(self.da*self.r,100)
        self.fc_dedim_rep = nn.Linear(self.da*self.r,100)
        #self.fc_gate = nn.Linear(self.hidden_size*2*self.r,self.r*self.hidden_size*2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
    def conv_block(self,input_seq,input_pos):
        h1_rep3 = self.conv1_3(input_seq.unsqueeze(1)).transpose(1,3).squeeze(1)
        h1_rep3_g = self.conv2_7(input_seq.unsqueeze(1)).squeeze(3)
        h2_rep3 = torch.bmm(self.sigmoid(h1_rep3_g), h1_rep3)
        
#         h1_rep5 = self.self.conv1_5(input_seq.unsqueeze(1)).transpose(1,3).squeeze(1)
#         h1_rep5_g = self.conv1_5(input_seq.unsqueeze(1)).squeeze(3)
#         h2_rep5 = torch.bmm(self.sigmoid(h1_rep1_g), h1_rep1)
        
#         h1_rep7 = self.conv1_3(input_seq.unsqueeze(1)).transpose(1,3).squeeze(1)
#         h1_rep7_g = self.conv2(input_seq.unsqueeze(1)).squeeze(3)
#         h2_rep7 = torch.bmm(self.sigmoid(h1_rep1_g), h1_rep1)
        return h2_rep3
    def forward(self,sentence1,sentence2,sen1_pos,sen2_pos,length1,length2,pad1_V,pad2_V,cuda_available = False):
        
#         h1 = torch.mean(self.embed(sentence1),1)
#         h1 =torch.squeeze(h1, dim=1
#         h2 = torch.mean(self.embed(sentence2),1)
#         h2 =torch.squeeze(h2,dim=1)
        input_1_pos = self.embed_pos(sen1_pos)
        input_2_pos = self.embed_pos(sen2_pos)
        
        input_1 = self.embed(sentence1)
        input_2 = self.embed(sentence2)
        if cuda_available:
            output_1 = torch.mul(pad1_V.float().cuda(),input_1)
            output_2 = torch.mul(pad2_V.float().cuda(),input_2)
        else:
            output_1 = torch.mul(pad1_V.float(),input_1)
            output_2 = torch.mul(pad2_V.float(),input_2)
        represent1 = self.conv_block(output_1,input_1_pos)
        represent2 = self.conv_block(output_2,input_2_pos)
        
        output1_new = represent1.view(-1,40000)
        output2_new = represent2.view(-1,40000)
        mul_represent = torch.mul(output1_new,output2_new)
        sub_represent = torch.abs(output1_new-output2_new)
        mul_dedim = self.fc_dedim_mul(mul_represent)
        sub_dedim = self.fc_dedim_sub(sub_represent)
        rep1_dedim = self.fc_dedim_rep(output1_new)
        rep2_dedim = self.fc_dedim_rep(output2_new)

        out_mul = self.relu(torch.cat((sub_dedim,mul_dedim,rep1_dedim,rep2_dedim),1))
        logits  = self.fc2(self.relu(self.fc1(out_mul)))
        return logits
    def init_weights(self,pretrained_weight):
        #init(self.embed.weight.data)
        initrange = 0.001
#         nn.init.uniform(self.embedding,a=0,b=0.00
        self.fc1.bias.data.fill_(0)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        #torch.nn.init.xavier_uniform(self.fc.weight.data, gain=math.sqrt(2.0))
        #self.embed.weight.data.normal_(mean=0, std=0.01)
        self.embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.fill_(0)
        
        self.fcWs1.weight.data.uniform_(-initrange, initrange)
        self.fcWs1.bias.data.fill_(0)
        self.fcWs2.weight.data.uniform_(-initrange, initrange)
        self.fcWs2.bias.data.fill_(0)
    def initHidden(self,batch_size):
        h0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
        c0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
        return h0,c0

In [ ]:
# class biRNN(nn.Module):
#     def __init__(self,vocab_size,embedding_size,hidden_size,output_size,n_layers = 1):
#         super(biRNN, self).__init__()
#         self.n_layers = n_layers
#         self.hidden_size = hidden_size
#         self.embedding_size = embedding_size
#         self.output_size =output_size
#         self.embed = nn.Embedding(vocab_size,embedding_size)
#         self.gru = nn.GRU(input_size=self.embedding_size,
#                             hidden_size=self.hidden_size,
#                             num_layers = self.n_layers,
#                             batch_first =True,
#                             bidirectional = True)
#         self.lstm = nn.LSTM(input_size=self.embedding_size,
#                             hidden_size=self.hidden_size,
#                             num_layers = self.n_layers,
#                             batch_first =True,
#                             bidirectional = True)
#         self.BN0 = nn.BatchNorm1d(4800)
#         self.BN1 = nn.BatchNorm1d(1200)
#         self.fc1 = nn.Linear(4*self.hidden_size*2*self.n_layers*2, 1200)
#         self.fc2 = nn.Linear(1200, 2)
#         self.relu = nn.ReLU()
#         self.softmax = nn.Softmax()
#     def forward(self,sentence1,sentence2,length1,length2,pad1_V,pad2_V,cuda_available = False):
# # 
#         input_1 = self.embed(sentence1)
#         input_2 = self.embed(sentence2)
# #         if cuda_available:
# #             output_1 = torch.mul(pad1_V.float().cuda(),input_1)
# #             output_2 = torch.mul(pad2_V.float().cuda(),input_2)
# #         else:
# #             output_1 = torch.mul(pad1_V.float(),input_1)
# #             output_2 = torch.mul(pad2_V.float(),input_2)
#         output_1,hidden1 = self.gru(input_1)
#         output_2,hidden2 = self.gru(input_2)
        
#         ls_o1,(ls_h1,ls_c1)= self.lstm(input_1)
#         ls_o2,(ls_h2,ls_c2) = self.lstm(input_2)
        
#         A1 = torch.transpose(hidden1, 0, 1)
#         B1 = torch.transpose(hidden2, 0, 1)
#         A1 = A1.contiguous().view(input_1.size()[0],-1)
#         B1 = B1.contiguous().view(input_2.size()[0],-1)
        
#         A2 = torch.transpose(ls_h1, 0, 1)
#         B2 = torch.transpose(ls_h2, 0, 1)
#         A2 = A2.contiguous().view(input_1.size()[0],-1)
#         B2 = B2.contiguous().view(input_2.size()[0],-1)
# #         output1 = torch.cat((output_1[:,length1,:self.hidden_size],output_1[i,0,-self.hidden_size:]), 0)
# #         output2 = torch.cat((output_2[:,length2,:self.hidden_size],output_2[i,0,-self.hidden_size:]), 0)
#         A = torch.cat((A1,A2), 1)
#         B = torch.cat((B1,B2), 1)
#         mul_represent = torch.mul(A,B)
#         sub_represent = torch.abs(A-B)
#         out_mul = self.relu(torch.cat((mul_represent,sub_represent,A,B),1))
#         logits  = self.fc2(self.BN1(self.relu(self.fc1(self.BN0(out_mul)))))
#         return logits
#     def init_weights(self,pretrained_weight):
#         #init(self.embed.weight.data)
#         initrange = 0.001
# #         nn.init.uniform(self.embedding,a=0,b=0.00
#         self.fc1.bias.data.fill_(0)
#         self.fc1.weight.data.uniform_(-initrange, initrange)
#         #torch.nn.init.xavier_uniform(self.fc.weight.data, gain=math.sqrt(2.0))
#         #self.embed.weight.data.normal_(mean=0, std=0.01)
#         self.embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
#         self.fc2.weight.data.uniform_(-initrange, initrange)
#         self.fc2.bias.data.fill_(0)
#     def initHidden(self,batch_size):
#         h0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
#         c0 = Variable(torch.zeros(2*self.n_layers,batch_size,self.hidden_size))
#         return h0,c0

In [ ]:
bilstm = biRNN_att(vocab_size = np.shape(embedding_matrix)[0],embedding_size = np.shape(embedding_matrix)[1],hidden_size = 300,output_size =2,n_layers =1)

In [ ]:
bilstm.init_weights(embedding_matrix)

In [ ]:
def get_batch(batch,batch_pos,hidden_num = 100,training = True):
    length1 = [len(sen1) for sen1 in batch[:,0]]
    length2 = [len(sen2) for sen2 in batch[:,1]]
    max_length1 = max(length1)
    max_length2 = max(length2)
    #print(max_length1)
    s1_batch = []
    s1_batch_pos = []
    s2_batch = []
    s2_batch_pos = []
    s1_batch_pad = []
    s2_batch_pad = []
    for sen,sen_pos in zip(batch,batch_pos):
        if len(sen[0])==max_length1:
            s1_batch.append(sen[0])
            s1_batch_pad.append(np.concatenate((np.ones([len(sen[0]),hidden_num]),
                                                np.zeros([max_length1-len(sen[0]),hidden_num])),0))
            s1_batch_pos.append(sen_pos[0])
        else:
            s1_batch.append(sen[0]+(max_length1-len(sen[0]))*[0])
            s1_batch_pos.append(sen_pos[0]+(max_length1-len(sen[0]))*[0])
            s1_batch_pad.append(np.concatenate((np.ones([len(sen[0]),hidden_num]),
                                                np.zeros([max_length1-len(sen[0]),hidden_num])),0))
        if len(sen[1])==max_length2:
            s2_batch.append(sen[1])
            s2_batch_pos.append(sen_pos[1])
            s2_batch_pad.append(np.concatenate((np.ones([len(sen[1]),hidden_num]),
                                                np.zeros([max_length2-len(sen[1]),hidden_num])),0))
        else:
            s2_batch.append(sen[1]+(max_length2-len(sen[1]))*[0])
            s2_batch_pos.append(sen_pos[1]+(max_length2-len(sen[1]))*[0])
            
            s2_batch_pad.append(np.concatenate((np.ones([len(sen[1]),hidden_num]),
                                                np.zeros([max_length2-len(sen[1]),hidden_num])),0))
    if training:   
        return [Variable(torch.from_numpy(np.array(s1_batch))),
                Variable(torch.from_numpy(np.array(s2_batch))),
                Variable(torch.from_numpy(np.array(s1_batch_pos))),
                Variable(torch.from_numpy(np.array(s2_batch_pos))),
                batch[:,2].astype(int),length1,length2,
                Variable(torch.from_numpy(np.array(s1_batch_pad))),
                Variable(torch.from_numpy(np.array(s2_batch_pad)))]
    else:
        return [Variable(torch.from_numpy(np.array(s1_batch))),
                Variable(torch.from_numpy(np.array(s2_batch))),
                Variable(torch.from_numpy(np.array(s1_batch_pos))),
                Variable(torch.from_numpy(np.array(s2_batch_pos))),
                length1,length2,
                Variable(torch.from_numpy(np.array(s1_batch_pad))),
                Variable(torch.from_numpy(np.array(s2_batch_pad)))]

In [ ]:
batch_s1,batch_s2,batch_s1_pos,batch_s2_pos,batch_label,length1_b,length2_b ,pad1,pad2= get_batch(np.array(data_index[0:20]),np.array(data_pos_index[0:20]),hidden_num=300,training=True)
#o = bilstm(batch_s1,batch_s2,length1_b,length2_b,pad1,pad2)

In [ ]:
o = bilstm(batch_s1,batch_s2,batch_s1_pos,batch_s2_pos,length1_b,length2_b,pad1,pad2)

In [ ]:
o.size()

In [ ]:
torch.cuda.set_device(1)

In [ ]:
learning_rate = 1e-3
criterior_cross = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(bilstm.parameters(), lr=1e-3)
criterior_cross.cuda()
bilstm.cuda()

In [ ]:
batch_dev_size = 128
dev_num = batch_dev_size*30
train_data_pos = data_pos_index[:-dev_num]
dev_data_pos = data_pos_index[-dev_num:]

train_data = data_index[:-dev_num]
dev_data = data_index[-dev_num:]
print('train_size: '+str(len(train_data))+' dev_size: '+str(len(dev_data)))

In [ ]:
batch_size =64
best_acc = 0
epoches = 2
best_loss = 10
num_model = 10
check_file_num = range(num_model)
step = 0
torch_softmax= nn.Softmax()
from sklearn.metrics import log_loss
for epoch in range(epoches):
    if epoch ==0:
        check_steps =250
    else:
        check_steps = 50
    #np.random.shuffle(train_data)
    for i in range(len(train_data)/batch_size):
        optimizer.zero_grad()
        batch_s1,batch_s2,batch_s1_pos,batch_s2_pos,batch_labels,length_s1,length_s2,pad1,pad2 = get_batch(np.array(train_data[i*batch_size:(i+1)*batch_size]),
                                                                                                           np.array(train_data_pos[i*batch_size:(i+1)*batch_size]),
                                                                                 hidden_num=300,training=True)
        if torch.cuda.is_available():
            o = bilstm(batch_s1.cuda(),batch_s2.cuda(),batch_s1_pos.cuda(),batch_s2_pos.cuda(),
                       length_s1,length_s2,pad1,pad2,cuda_available=True )  
        loss = criterior_cross(o, Variable(torch.from_numpy(batch_labels)).cuda())
        loss.backward()
        optimizer.step()
        if (i+1) % check_steps ==0:
            count_correct =0
            allsum =[]
            pred_dev_batch = []
            true_labels_dev = []
            for num in range(len(dev_data)/batch_dev_size):
                batch_s1_dev,batch_s2_dev,batch_s1_dev_pos,batch_s2_dev_pos,batch_labels_dev,length_s1_dev,length_s2_dev,pad1dev,pad2dev = get_batch(np.array(dev_data[num*batch_dev_size:(num+1)*batch_dev_size]),
                                                                                                                   np.array(dev_data_pos[num*batch_dev_size:(num+1)*batch_dev_size]),
                                                                                                                   hidden_num=300)
                o = bilstm(batch_s1_dev.cuda(),batch_s2_dev.cuda(),batch_s1_dev_pos.cuda(),batch_s2_dev_pos.cuda(),
                           length_s1_dev,length_s2_dev,pad1dev,pad2dev,cuda_available=True)
                out_model = torch_softmax(o)
                allsum.append(sum(out_model.data.max(1)[1].view(-1).cpu()))
                count_correct += sum(out_model.data.max(1)[1].view(-1).cpu().numpy()==batch_labels_dev)#.cuda().long()).long().sum().data[0]
                pred_dev_batch.extend(out_model.data.cpu().numpy()[:,1].astype(float))
                #bi_loss = binary_log_loss(o.data[:,1],torch.from_numpy(batch_labels_dev).cuda())
                true_labels_dev.extend(batch_labels_dev)
                #print(bi_loss)
            acc = count_correct/(len(dev_data)+0.0)
            current_loss = log_loss(true_labels_dev, pred_dev_batch)
            if current_loss<best_loss:
                best_loss = current_loss
                path = '/home/zhangjinbin/research/quora/model_restore/cnn_model'+str(check_file_num[step]%num_model)+'.pkl'
                torch.save(bilstm.state_dict(), path)
                print("save in : "+ path)
                step = step +1
                if step==num_model:
                    step = 0
            print('step: '+str(i) +' acc: '+str(acc)+' '+ str(sum(allsum))+" loss: "+str(current_loss))